Perfor convergence test in the L2 norm of the classical state vector, for strong and weak data. You need simulations of the same setup, in three different resolutions (radial grid points). You need to save "data",

In [ ]:
# load packages
using HDF5
using LaTeXStrings
using Plots; pythonplot()
using DelimitedFiles
using SpheriCo
using ProgressBars

In [ ]:
# give the directory where the data from all the runs are saved
dir = "../examples/classical_runs/"
par1 = "a0.5_b5.0_c2.0_rmax30_tmax15_cfl0.125_sigma0.02_infalling_rmax_false_rand_false_overMp2_25.132741228718345_damp0"
par2 = "a0.05_b5.0_c2.0_rmax30_tmax15_cfl0.125_sigma0.02_infalling_rmax_false_rand_false_overMp2_25.132741228718345_damp0"
your_dir1 = dir*par1
your_dir2 = dir*par2

# make directory where the plots will be saved
out_dir = "./convergence_plots/norms/"
if ispath(out_dir)==false
    mkpath(out_dir)
end

In [ ]:
# which are the resolutions? (in terms of D, where Nr = 128*2^D + 3)
resols = [3,4,5]

# initiate l2 matrices for calculations later
# coarse resol
Dc = resols[1]
Nrc = 128*2^Dc + 3
# list all available iterations (and corresponding files)
(its_c1, all_filenames_c1) = list_h5_files(your_dir1*"/data_$(Nrc)", prefix="data_")
(its_c2, all_filenames_c2) = list_h5_files(your_dir2*"/data_$(Nrc)", prefix="data_")

# the l2 norm of the diff between two resols, on the grid points of the coarse resol
l2_diffs_strong  = zeros( (length(resols)-1, length(its_c1) ))
l2_diffs_weak  = zeros( (length(resols)-1, length(its_c2) ))
# timesteps of the coarse resol; common in all resols (tuned like that in the simulation)
t_lists_strong = zeros( (length(resols)-1, length(its_c1) ))
t_lists_weak = zeros( (length(resols)-1, length(its_c2) ))
# the time when AH forms (if any) in coarse resol.
t_AH = 0.0;

In [ ]:
# take diff between two state vector on the common points with the coarsest resol
function v_diff(your_dir::String, D::Int, i::Int, its_1::Array, its_2::Array)
    it_1 = its_1[i]
    it_2 = its_2[i]
    it_str_1  = lpad(it_1, 4, "0")
    it_str_2  = lpad(it_2, 4, "0")
    
    Nr1 = 128*2^D + 3
    Nr2 = 128*2^(D+1) + 3
    
    v1 = h5read(your_dir*"/data_$(Nr1)/data_$(it_str_1).h5","v")
    v2 = h5read(your_dir*"/data_$(Nr2)/data_$(it_str_2).h5","v")
    
    t1 = h5readattr(your_dir*"/data_$(Nr1)/data_$(it_str_1).h5", "/")["time"]
    t2 = h5readattr(your_dir*"/data_$(Nr2)/data_$(it_str_2).h5", "/")["time"]
    
    r1 = h5read(your_dir*"/data_$(Nr1)/r.h5","r")
    r2 = h5read(your_dir*"/data_$(Nr2)/r.h5","r")

    # make sure we're comparing the same timestep
    @assert t1 ≈ t2

    # take diff only on the common points of the coarsest resol
    step1 = (D - Dc)
    step2 = (D + 1 - Dc)

    # check the common grid points
    @assert r1[3:2^step1:end] ≈ r2[3:2^step2:end]
    
    v12 = v1[3:2^step1:end,:] - v2[3:2^step2:end,:]
    return v12
end

In [ ]:
# get index of r_c in coarse resol, for which  r[index] is the first causally connected point
function find_ri_max_causal(your_dir::String, Nr::Int, its_c::Array)
    it_c      = its_c[end]
    it_str_c  = lpad(it_c, 4, "0")
    tc_end    =  h5readattr(your_dir*"/data_$(Nr)/data_$(it_str_c).h5", "/")["time"]
    ii = 1
    r_c = h5read(your_dir*"/data_$(Nr)/r.h5","r")
    while r_c[ii] != r_c[end] - tc_end
        ii = ii + 1
    end
    ri_max_causal = ii-1
    drc = r_c[4] - r_c[3]
    r_c[ri_max_causal] == r_c[end] - tc_end - drc
    return ri_max_causal
end

ri_max_causal1 = find_ri_max_causal(your_dir1, Nrc, its_c1)
ri_max_causal2 = find_ri_max_causal(your_dir2, Nrc, its_c2)
@assert ri_max_causal1 ≈ ri_max_causal2
ri_max_causal = ri_max_causal1

println("ri_max_causal=",ri_max_causal)

rc = h5read(your_dir1*"/data_$(Nrc)/r.h5","r")
drc = rc[4] - rc[3] # used later in norms
println("rc[ri_max_causal]=",rc[ri_max_causal])


In [ ]:
# get time that AH first appears
function find_t_AH(your_dir::String, Nr::Int, its_c::Array)
    j = 1
    it_c = its_c[j]
    it_str_c  = lpad(it_c, 4, "0")

    vc    = h5read(your_dir*"/data_$(Nr)/data_$(it_str_c).h5","v")
    tc    = h5readattr(your_dir*"/data_$(Nr)/data_$(it_str_c).h5", "/")["time"]
    r_c = h5read(your_dir*"/data_$(Nr)/r.h5","r")
    Ac    = vc[:,4]
    Bc    = vc[:,5]
    KBc   = vc[:,9]
    AH    = find_AH(r_c, Bc, Ac, KBc)

    while AH < 0
        it_c = its_c[j]
        it_str_c  = lpad(it_c, 4, "0")
        vc = h5read(your_dir*"/data_$(Nr)/data_$(it_str_c).h5","v")
        tc = h5readattr(your_dir*"/data_$(Nr)/data_$(it_str_c).h5", "/")["time"]
        Ac    = vc[:,4]
        Bc    = vc[:,5]
        KBc   = vc[:,9]
        AH = find_AH(r_c, Bc, Ac, KBc)
        j = j + 1  
    end
    t_AH = tc
    return t_AH
end

t_AH = find_t_AH(your_dir1, Nrc, its_c1)
println("t_AH = ",t_AH)

In [ ]:
println("strong data: calculate norms")
for Di in ProgressBar(1:length(resols)-1)
    D = resols[Di]
    Nr1 = 128*2^D + 3 # coarse
    Nr2 = 128*2^(D+1) + 3 # medium
    
    # list all available iterations (and corresponding files)
    (its_1, all_filenames_1) = list_h5_files(your_dir1*"/data_$(Nr1)", prefix="data_")
    (its_2, all_filenames_2) = list_h5_files(your_dir1*"/data_$(Nr2)", prefix="data_")
    
    # it starts from r_c[3], so shift the ri_max_causal by 3
    for ti ∈ 1:length(its_c1)
        v_12 = v_diff(your_dir1, D, ti, its_1, its_2)
        it_1       = its_1[ti]
        it_str_1   = lpad(it_1, 4, "0")
        t_lists_strong[Di, ti]  =  h5readattr(your_dir1*"/data_$(Nr1)/data_$(it_str_1).h5", "/")["time"]
        l2_diffs_strong[Di, ti] = sqrt(sum(v_12[1:ri_max_causal-3].^2)*drc)
    end
end

In [ ]:
println("weak data: calculate norms")
for Di in ProgressBar(1:length(resols)-1)
    D = resols[Di]
    Nr1 = 128*2^D + 3 # coarse
    Nr2 = 128*2^(D+1) + 3 # medium
    
    # list all available iterations (and corresponding files)
    (its_1, all_filenames_1) = list_h5_files(your_dir2*"/data_$(Nr1)", prefix="data_")
    (its_2, all_filenames_2) = list_h5_files(your_dir2*"/data_$(Nr2)", prefix="data_")
    
    # it starts from r_c[3], so shift the ri_max_causal by 3
    for ti ∈ 1:length(its_c2)
        v_12 = v_diff(your_dir2, D, ti, its_1, its_2)
        it_1       = its_1[ti]
        it_str_1   = lpad(it_1, 4, "0")
        t_lists_weak[Di, ti]  =  h5readattr(your_dir2*"/data_$(Nr1)/data_$(it_str_1).h5", "/")["time"]
        l2_diffs_weak[Di, ti] = sqrt(sum(v_12[1:ri_max_causal-3].^2)*drc)
    end
end

In [ ]:
plot(t_lists_strong[1,2:end], l2_diffs_strong[1,2:end],
    frame=true, linewidth=2.5, label=L"||\mathbf{u}_{cm}||")

plot!(t_lists_strong[1,2:end], 4*l2_diffs_strong[2,2:end],
    frame=true, linewidth=3, style=:dash, label=L"4 \, || \mathbf{u}_{mf}||", legend=:bottomright, 
    legendfontsize=10,
   # ylabel="norms",
    ytickfont=10, yguidefontsize=10,
    xtickfont=10, xguidefontsize=10,
    xlabel="time",
    #yscale=:log10,
)

p1 = plot!([t_AH], seriestype="vline", label=L"t_{AH}", linewidth=1, color = "black", title="supercritical",
    xlim=(t_lists_strong[1,2],t_lists_strong[1,end]),
    xticks=([0, 4, 8, 12], ["0","4","8","12"]),
    yticks=([0, 5*1e-5, 1e-4, 1.5*1e-4], [L"0",L"10^{-5}",L"10^{-4}",L"1.5 \cdot 10^{-4}"])
)

plot(t_lists_weak[1,2:end], l2_diffs_weak[1,2:end],
    frame=true, linewidth=2.5, label=L"||\mathbf{u}_{cm}||")

p2 = plot!(t_lists_weak[1,2:end], 4*l2_diffs_weak[2,2:end],
        frame=true, linewidth=3, style=:dash, label=L"4 \, || \mathbf{u}_{mf}||",
        legend=false, legendfontsize=10,
        xlabel="time", xtickfont=10, xguidefontsize=10,
        #ylabel="norms",
        ytickfont=10, yguidefontsize=10, title="subcritical",
        xlim=(t_lists_weak[1,2],t_lists_weak[1,end]),
        #yscale=:log10,
      # xticks=([0,5*1e-5,1e-4,1.5*1e-4],["","","",""]),  
    xticks=([0, 4, 8, 12], ["0","4","8","12"]),
    yticks=([0, 5*1e-5, 1e-4, 1.5*1e-4], ["","","",""])

)

plt = plot(p1, p2, layout = grid(1,2), wsize = (800,230))


savefig(plt, out_dir*"/L2_smooth_strong_weak.pdf")
